In [1]:
import pandas as pd
import re
import os
from rusenttokenize import ru_sent_tokenize
import itertools
from pymorphy2 import MorphAnalyzer
import collections
from collections import Counter, defaultdict
import numpy as np
from string import punctuation
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
import nltk
import pickle
import configparser
from deeppavlov.models.embedders.elmo_embedder import ELMoEmbedder

from tqdm import tqdm
from copy import deepcopy
from PIL import Image 
import pytesseract 
import sys 
from pdf2image import convert_from_path 
import glob

morph = MorphAnalyzer()
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/julia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/julia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/julia/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/julia/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
[nltk_data] Downloading package stopwords to /home/julia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
class ContractRelation:
    def __init__(self):
        self.elmo = ELMoEmbedder("http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-news_wmt11-16_1.5M_steps.tar.gz")

        # pickle_path = './dog_data'
        pickle_path = './'
        
        dog_file = open(os.path.join(pickle_path, 'dog_articles.pkl'), 'rb')
        self.index2article = pickle.load(dog_file)
        dog_file.close()
        
        emb_file = open(os.path.join(pickle_path, 'dog_embs_elmo.pkl'), 'rb')
        self.index2emb = pickle.load(emb_file)
        emb_file.close()
    
    @staticmethod
    def normalize(text):
        stops = set(stopwords.words('russian'))
        normalized_text = text.replace('\n', ' ')
        normalized_text = re.sub('(?:(?:https?):\/\/)?[\w/\-?=%.]+\.[\w/\-?=%.]+', '', normalized_text)
        wordpunct_tokenize = WordPunctTokenizer().tokenize
        normalized_text = wordpunct_tokenize(normalized_text)
        normalized_text = [word.replace('№', '') for word in normalized_text]
        normalized_text = [re.sub(r'\d+(?:\.\d+)?', '', word)  for word in normalized_text]
        normalized_text = [morph.parse(word.lower())[0].normal_form for word \
                                            in normalized_text if word not in punctuation+'«»']
        normalized_text = [word.strip(punctuation) for word in normalized_text if word not in stops]
        normalized_text = ' '.join(normalized_text)
        normalized_text = re.sub(' +', ' ', normalized_text)

        return normalized_text
    
    def predict(self, text):
        text = self.normalize(text)
        emb_tgt = self.elmo([text.split()])
        
        dic = {}
        for key, value in self.index2emb.items():
            score = np.inner(value, emb_tgt) / (
                        np.linalg.norm(value) * (np.linalg.norm(emb_tgt)))
            dic.update({score[0][0]: key})
        od = collections.OrderedDict(sorted(dic.items(), reverse=True))
        if list(od.keys())[0] > 0.7:
            label = list(od.values())[0]
            return self.index2article[label][0][0]
        else:
            return None

## Test

In [3]:
# %%time
# pages = convert_from_path('./correspondencesemanticanalysis/sample.pdf', 500) 
# # pages = convert_from_path('sample.pdf', 500)
# image_counter = 1
# text_full = []

# for page in pages: 
#     text = str(((pytesseract.image_to_string(page, lang="rus+eng"))))
#     image_counter += 1
#     text_full.append(text)

In [4]:
# text = ' '.join(text_full)
# text

In [5]:
# start = 'Уважаемый'
# end = 'С уважением'
# start_i = text.find(start)
# end_i = text.find(end)
# text = text[start_i:end_i]
# text

In [4]:
annotated = pd.read_json('annotation.json')
annotated.head()

,id,markups
0,накладная_на_передачу_19.06.2019_06_55_09_texted,"[{'markup_id': 2145, 'user': '5Maria_Denisova'..."
1,!!Аккредитованные_МО_texted,"[{'markup_id': 2061, 'user': '5Maria_Denisova'..."
2,14_6120_от_25.12.19_в_КХМ_Бланк_письма_БГП_о_с...,"[{'markup_id': 2055, 'user': '5Maria_Denisova'..."
3,14-3443_от_22.08.2019_texted,"[{'markup_id': 2051, 'user': '5Maria_Denisova'..."
4,14-05-8736_от_05.08.19_о_Согласовании_Субподря...,"[{'markup_id': 2049, 'user': '5Maria_Denisova'..."


In [5]:
annotated['markups'][10]

[{'markup_id': 2158,
  'user': '5Maria_Denisova',
  'escape': [{'text': '23.02.2020', 'tags': ['1сроки']},
   {'text': 'Приложение 1: исполнительная геодезическая схема №2 фактической отсыпки площадки',
    'tags': ['34  ПРИЛОЖЕНИЯ К ДОГОВОРУ']}]}]

In [12]:
clf = ContractRelation()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [21]:
for pair in zip(annotated['id'].tolist(), annotated['markups'].tolist()):
    file = open('./pdf_texted/'+pair[0]+'.txt', 'r')
    text = file.read()
    if 'Уважаемый' in text:
        start = 'Уважаемый'
        end = 'С уважением'
        start_i = text.find(start)
        end_i = text.find(end)
        text = text[start_i:end_i]
    # print(text)
    text = text.replace('\n', ' ')
    normalized_text = re.sub('(?:(?:https?):\/\/)?[\w/\-?=%.]+\.[\w/\-?=%.]+', '', text)
    normalized_text = re.sub(r'\d+(?:\.\d+)?', '', normalized_text)
    sentences = ru_sent_tokenize(normalized_text)
    for sentence in sentences:
        label = clf.predict(sentence)
        if label != None:
            print('-'*90)
            print(sentence)
            print('-'*90)
            print('ПУНКТ ДОГОВОРА: ', label)
            print()
    dics = pair[1]
    for dic in dics:
        print(dic['escape'])
    print('='*80)

------------------------------------------------------------------------------------------
тел.:---- (звонок по России бесплатный)     СПЕЦСВЯЗЬ № накладной |  экспресс Федеральное государственное унитарное предприятие Дата "Главный центр специальной связи" заполнения --    Заполняется отправителем ( -слойный бланк, пишите с нажимом)  говор № ООО  .
------------------------------------------------------------------------------------------
ПУНКТ ДОГОВОРА:  СТАТЬЯ 13. ОБЩИЕ УСЛОВИЯ ПРОИЗВОДСТВА РАБОТ. НАРЯД-ДОПУСКИ ( п.п. 13.1 13.4)

------------------------------------------------------------------------------------------
[ Оплата наличными [|]   ОТПРАВЛЕНИЙ  _ Ро [roms Время   О ВРУЧЕНИИ ОТПРАВЛЕНИЯ                     Способ вручения: LJ лично L через доверенное лицо     Служебные отметки:  Документы, подтверждающие личность представителя получателя, проверены  рт Время  A подтверждаю, что отправление принято в закрытом виде, отсутствуют внешние повреждения упаковки, перевязи, печатей

[]
[{'text': 'Об аккредитации МО', 'tags': ['2 предмет договора']}]
[]
------------------------------------------------------------------------------------------
После подписания Сторонами Соглашения о перемене лиц (переуступка Заказчика от ООО «ГПН-Ямала» в ООО «ГПН-Развитие») по Договору № ГНЯ- ///Р/-ОЗ/ГНР-///Р/ от  на строительство объекта: «Комплекса объектов по компримированию и подготовке газа и конденсата Новопортовского нефтегазоконденсатного месторождения» в адрес АО «ТРЕСТ КХМ» был направлен (исх.№ / от ) проект дополнительного соглашения в части внесения изменений в «Соглашение в области ПЭТ, ОТ и ГЗ» (Приложение № к Договору).
------------------------------------------------------------------------------------------
ПУНКТ ДОГОВОРА:  СТАТЬЯ 14. ТРЕБОВАНИЯ В ОБЛАСТИ ПРОМЫШЛЕННОЙ И ЭКОЛОГИЧЕСКОЙ БЕЗОПАСНОСТИ, ОХРАНЫ ТРУДА И ГРАЖДАНСКОЙ ЗАЩИТЫ (ПЭБ, ОТ и ГЗ) ПРИ ВЫПОЛНЕНИИ РАБОТ НА ОБЪЕКТАХ ЗАКАЗЧИКА ( п.п. 14.1 14.6)



[{'text': '(Приложение №13 к Договору).', 'tags': ['34  ПРИЛОЖЕНИЯ К ДОГОВОРУ']}, {'text': '«Соглашение в области ПЭТ, ОТ и ГЗ»', 'tags': ['14  ТРЕБОВАНИЯ В ОБЛАСТИ ПРОМЫШЛЕННОЙ И ЭКОЛОГИЧЕСКОЙ БЕЗОПАСНОСТИ, ОХРАНЫ ТРУДА И ГРАЖДАНСКОЙ ЗАЩИТЫ (ПЭБ, ОТ и ГЗ) ПРИ ВЫПОЛНЕНИИ РАБОТ НА ОБЪЕКТАХ ЗАКАЗЧИКА']}, {'text': 'Просим сообщить сроки принятия решения или направить в наш адрес проект Соглашения, подписанного с Вашей стороны.', 'tags': ['6 сроки действия договора и сроки выполнения работ']}, {'text': 'В рабочем порядке неоднократно запрашивались и переносились сроки рассмотрения.', 'tags': ['6 сроки действия договора и сроки выполнения работ']}, {'text': '22.08.2019', 'tags': ['1сроки']}, {'text': '05.06.2019', 'tags': ['1сроки']}, {'text': 'О сроках рассмотрения Дополнительного соглашения', 'tags': ['2 предмет договора']}]
------------------------------------------------------------------------------------------
Направляем Вам проект дополнительного соглашения №  к договору №ГНЯ- ///Р/-

[{'text': 'О направлении проекта дополнительного соглашения №1', 'tags': ['2 предмет договора']}, {'text': '05.06.2019', 'tags': ['1сроки']}]
[{'text': 'Проект дополнительного соглашения № 1', 'tags': ['34  ПРИЛОЖЕНИЯ К ДОГОВОРУ']}, {'text': 'О направлении проекта дополнительного соглашения №1', 'tags': ['2 предмет договора']}]
[{'text': 'Соглашение в области производственной безопасности', 'tags': ['15МОТИВАЦИЯ СОБЛЮДЕНИЯ ПОЛОЖЕНИЙ ДОГОВОРА В ОБЛАСТИ ПБ']}, {'text': 'Перечень нарушений и штрафных санкций', 'tags': ['7 заверения об обстоятельствах, имеющих значение для заключения, исполнения или прекращения договора']}, {'text': 'изменения требований Компании в области промышленной безопасности', 'tags': ['15МОТИВАЦИЯ СОБЛЮДЕНИЯ ПОЛОЖЕНИЙ ДОГОВОРА В ОБЛАСТИ ПБ']}]
------------------------------------------------------------------------------------------
Рассмотрев направленные Вами материалы в отношении ООО "СПЕКТР- ЭЛЕКТРО", ИНН , сообщаем, что в рамках договора № ГНЯ-///Р/-  от  на в

[{'text': 'в январе 2020г', 'tags': ['1сроки']}, {'text': '14.01.2020г', 'tags': ['1сроки']}, {'text': 'качества', 'tags': ['1качество']}]
[{'text': 'по объекту «Комплекс объектов по компримированию и подготовке газа и конденсата (УКПГ) Новопортовского нефтегазоконденсатного месторождения», в январе 2020г авансирование на мобилизацию не требуется. ', 'tags': ['2 предмет договора']}]
------------------------------------------------------------------------------------------
Направляем подписанный Обществом Договор №ГНЯ-///Р/- от  на выполнение работ по строительству объекта: «Комплекса объектов по компримированию и подготовке газа и конденсата (УКПГ) Новопортовского нефтегазоконденсатного месторождения».
------------------------------------------------------------------------------------------
ПУНКТ ДОГОВОРА:  СТАТЬЯ 11. РИСКИ СЛУЧАЙНОЙ ГИБЕЛИ ИЛИ ПОВРЕЖДЕНИЯ ОБЪЕКТА ( п.п. 11.1 11.1)

[{'text': '05.06.2019 г.', 'tags': ['1сроки']}, {'text': 'Приложение: Договор № ГНЯ-19/11024/00865/Р/9-

KeyboardInterrupt: 

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [9]:
clf.predict(text)

'СТАТЬЯ 15. МОТИВАЦИЯ СОБЛЮДЕНИЯ ПОЛОЖЕНИЙ ДОГОВОРА В ОБЛАСТИ ПБ ( п.п. 15.1 15.18)'